In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch
import copy
import mlflow
from pathlib import Path

In [ ]:
from src.training.SmallRawDataset import SmallRawDataset

from src.training.losses.ShadowAwareLoss import ShadowAwareLoss
from src.training.VGGFeatureExtractor import VGGFeatureExtractor
from src.training.utils import apply_gamma_torch
from src.training.train_loop import visualize
from src.training.load_config import load_config


In [ ]:
run_config = load_config()
dataset_path = Path(run_config['jpeg_output_subdir'])
align_csv = dataset_path / run_config['align_csv']
device=run_config['device']

batch_size = run_config['batch_size']
lr = run_config['lr_base'] * batch_size
clipping =  run_config['clipping']

num_epochs = run_config['num_epochs_pretraining']
val_split = run_config['val_split']
crop_size = run_config['crop_size']
experiment = run_config['mlflow_experiment']
model_params = run_config['model_params']
rggb = model_params['rggb']

In [ ]:

RUN_ID = "66f11f4639f24e9ea75b4c953147be15"  
ARTIFACT_PATH = run_config['run_path']

model_uri = f"runs:/{RUN_ID}/{ARTIFACT_PATH}"

try:
    model = mlflow.pytorch.load_model(model_uri)
    model.eval()
    print(f"Model successfully loaded from MLflow URI: {model_uri}")
    

except Exception as e:
    print(f"Error loading model from MLflow: {e}")

In [ ]:
dataset = SmallRawDataset(dataset_path, align_csv, crop_size=crop_size)

# Split dataset into train and val
val_size = int(len(dataset) * val_split)
train_size = len(dataset) - val_size
torch.manual_seed(42)  # For reproducibility
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
# Set the validation dataset to use the same crops
val_dataset = copy.deepcopy(val_dataset)
val_dataset.dataset.validation = True

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

vfe = VGGFeatureExtractor(config=((1, 64), (1, 128), (1, 256), (1, 512), (1, 512),), 
                          feature_layers=[14], 
                          activation=nn.ReLU
                          )
vfe = vfe.to(device)

loss_fn = ShadowAwareLoss(
    alpha=run_config['alpha'],
    beta=run_config['beta'],
    l1_weight=run_config['l1_weight'],
    ssim_weight=run_config['ssim_weight'],
    tv_weight=run_config['tv_weight'],
    vgg_loss_weight=run_config['vgg_loss_weight'],
    apply_gamma_fn=apply_gamma_torch,
    vgg_feature_extractor=vfe,
    device=device,
)

In [ ]:
import numpy as np
subset_indices = np.array(val_dataset.indices)  # indices in the original dataset
mask = val_dataset.dataset.df.iso.values[subset_indices] == 65535
matching_indices_in_subset = np.nonzero(mask)[0]
matching_indices_in_subset

In [ ]:
visualize(matching_indices_in_subset,  model, val_dataset, device, loss_fn, rggb=rggb)